In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
! export TF_CPP_MIN_LOG_LEVEL=2

In [3]:
# gpu check 
!nvidia-smi

Thu Apr 11 16:16:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:3A:00.0 Off |                    0 |
| N/A   29C    P0              40W / 300W |      9MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

## Environment Prep

In [5]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [6]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
# from tensorflow.keras.models import Model
from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
from transformers import AutoTokenizer, pipeline, AutoModel
import resources.smart_cond as sc
# from google.colab import files
from keras.utils import pad_sequences

## Version Check

In [7]:
import tensorflow as tf
print(tf. __version__)

pickle.format_version

2.12.0


'4.0'

## Load Data

In [8]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
del pkl

In [9]:
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date: [**2119-5-4**] D...,noteevents,1.000000,1.000000
1,18407,28.016667,Text,Admission Date: [**2112-12-8**] ...,noteevents,1.000000,1.000000
2,40300,155.166667,Text,Admission Date: [**2194-7-18**] ...,noteevents,1.000000,1.000000
3,23747,52.383333,Text,Admission Date: [**2194-1-7**] D...,noteevents,1.000000,1.000000
4,2357,73.133333,Text,Admission Date: [**2186-6-7**] Discharge ...,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [10]:
data.loc[data['variable'] == 'Text', 'value'] = 1.0
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,1.0,noteevents,1.000000,1.000000
1,18407,28.016667,Text,1.0,noteevents,1.000000,1.000000
2,40300,155.166667,Text,1.0,noteevents,1.000000,1.000000
3,23747,52.383333,Text,1.0,noteevents,1.000000,1.000000
4,2357,73.133333,Text,1.0,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [11]:
pred_window = 2 # hours
obs_windows = range(20, 124, 4)
# Remove test patients.
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(test_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(test_sub)]
data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# Fix age.
data.loc[(data.variable=='Age')&(data.value>200), 'value'] = 91.4
# data[data.variable=='Age'][data.value>200]['value'] = 91.4

In [12]:
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds
# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
fore_times_ip = []
fore_values_ip = []
fore_varis_ip = []
fore_op = []
fore_inds = []
def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask
def pad(x):
    return x+[0]*(fore_max_len-len(x))
for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)    
    obs_data = data.loc[(data.hour<w)&(data.hour>=w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind':list, 'hour':list, 'value':list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_op.append(np.array(list(obs_data.vind_value)))
    fore_inds.append(np.array(list(obs_data.ts_ind)))
    fore_times_ip.append(np.array(list(obs_data.hour)))
    fore_values_ip.append(np.array(list(obs_data.value)))
    fore_varis_ip.append(np.array(list(obs_data.vind)))
del data
fore_times_ip = np.concatenate(fore_times_ip, axis=0)
fore_values_ip = np.concatenate(fore_values_ip, axis=0)
fore_varis_ip = np.concatenate(fore_varis_ip, axis=0)
fore_op = np.concatenate(fore_op, axis=0)
fore_inds = np.concatenate(fore_inds, axis=0)
fore_demo = demo[fore_inds]
# Get train and valid ts_ind for forecast task.
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
rem_sub = oc.loc[~oc.SUBJECT_ID.isin(np.concatenate((train_ind, valid_ind)))].SUBJECT_ID.unique()
bp = int(0.8*len(rem_sub))
train_sub = np.concatenate((train_sub, rem_sub[:bp]))
valid_sub = np.concatenate((valid_sub, rem_sub[bp:]))
train_ind = oc.loc[oc.SUBJECT_ID.isin(train_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
valid_ind = oc.loc[oc.SUBJECT_ID.isin(valid_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
# Generate 3 sets of inputs and outputs.
train_ind = np.argwhere(np.in1d(fore_inds, train_ind)).flatten()
valid_ind = np.argwhere(np.in1d(fore_inds, valid_ind)).flatten()
fore_train_ip = [ip[train_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
fore_valid_ip = [ip[valid_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
del fore_times_ip, fore_values_ip, fore_varis_ip, demo, fore_demo
fore_train_op = fore_op[train_ind]
fore_valid_op = fore_op[valid_ind]
del fore_op

91626it [00:00, 800957.25it/s]
100%|██████████| 26/26 [05:06<00:00, 11.79s/it]


In [13]:
# add text features
text_ip = pickle.load(open('Data/hierar/text_emb_input_train_val_1.pkl', 'rb'))

train_text_ip = text_ip[0]
# valid_text_ip = text_ip[1]

fore_train_ip.append(train_text_ip)
# fore_valid_ip.append(valid_text_ip)

In [14]:
# train text times
train_text_times = []
train_text_varis = []

train_times = fore_train_ip[1]
train_varis = fore_train_ip[3]

for i in tqdm(range(len(fore_train_ip[0]))):
    times = []
    varis = []
    for j in range(880):
        if train_varis[i][j] == 124:
            # times += [train_times[i][j]] * 768
            times.append(train_times[i][j])
            varis.append(135)
    train_text_times.append(np.array(times))
    train_text_varis.append(np.array(varis))

train_text_times = np.array(train_text_times)
train_text_varis = np.array(train_text_varis)

100%|██████████| 449482/449482 [02:16<00:00, 3288.56it/s]
/scratch/slurm_tmpdir/job_23365184/ipykernel_562684/1196284020.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_text_times = np.array(train_text_times)
/scratch/slurm_tmpdir/job_23365184/ipykernel_562684/1196284020.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_text_varis = np.array(train_text_varis)


In [ ]:
# padding
padded_train_text_times = pad_sequences(train_text_times, maxlen=50, padding='post', dtype='float32')
padded_train_text_varis = pad_sequences(train_text_varis, maxlen=50, padding='post')

del train_text_times, train_text_varis, train_times, train_varis

In [ ]:
# # valid text times
# valid_text_times = []
# valid_text_varis = []

# valid_times = fore_valid_ip[1]
# valid_varis = fore_valid_ip[3]

# for i in tqdm(range(len(fore_valid_ip[0]))):
#     times = []
#     varis = []
#     for j in range(880):
#         if valid_varis[i][j] == 124:
#             # times += [valid_times[i][j]] * 768
#             times.append(valid_times[i][j])
#             varis.append(135)
#     valid_text_times.append(np.array(times))
#     valid_text_varis.append(np.array(varis))

# valid_text_times = np.array(valid_text_times)
# valid_text_varis = np.array(valid_text_varis)

In [ ]:
# # padding
# padded_valid_text_times = pad_sequences(valid_text_times, maxlen=50, padding='post', dtype='float32')
# padded_valid_text_varis = pad_sequences(valid_text_varis, maxlen=50, padding='post')

# del valid_text_times, valid_text_varis, valid_times, valid_varis

In [ ]:
fore_train_ip.append(padded_train_text_times)
fore_train_ip.append(padded_train_text_varis)
# fore_valid_ip.append(padded_valid_text_times)
# fore_valid_ip.append(padded_valid_text_varis)

In [ ]:
def get_res(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    minrp = np.minimum(precision, recall).max()
    roc_auc = roc_auc_score(y_true, y_pred)
    return [roc_auc, pr_auc, minrp]

# ######################################################################################################## 
# ######################################################################################################## 
# class_weights = compute_class_weight(class_weight='balanced', classes=[0,1], y=train_op)
# def mortality_loss(y_true, y_pred):
#     sample_weights = (1-y_true)*class_weights[0] + y_true*class_weights[1]
#     bce = K.binary_crossentropy(y_true, y_pred)
#     return K.mean(sample_weights*bce, axis=-1)
# ######################################################################################################## 
# ######################################################################################################## 

# var_weights = np.sum(fore_train_op[:, V:], axis=0)
# var_weights[var_weights==0] = var_weights.max()
# var_weights = var_weights.max()/var_weights
# var_weights = var_weights.reshape((1, V))
def forecast_loss(y_true, y_pred):
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

def get_min_loss(weight):
    def min_loss(y_true, y_pred):
        return weight*y_pred
    return min_loss

class CustomCallback(Callback):
    def __init__(self, validation_data, batch_size):
        self.val_x, self.val_y = validation_data
        self.batch_size = batch_size
        super(Callback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.val_x, verbose=0, batch_size=self.batch_size)
        if type(y_pred)==type([]):
            y_pred = y_pred[0]
        precision, recall, thresholds = precision_recall_curve(self.val_y, y_pred)
        pr_auc = auc(recall, precision)
        roc_auc = roc_auc_score(self.val_y, y_pred)
        logs['custom_metric'] = pr_auc + roc_auc
        print ('val_aucs:', pr_auc, roc_auc)

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Activation, Dropout, Softmax, Layer, InputSpec, Input, Dense, Lambda, TimeDistributed, Concatenate, Add
from tensorflow.keras import initializers, regularizers, constraints, Model
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow import nn

In [ ]:
# # SEFT positional encoding
# class PositionalEncoding(tf.keras.layers.Layer):
#     def __init__(self, max_time=20000, n_dim=10, **kwargs):
#         self.max_time = max_time
#         self.n_dim = n_dim
#         self._num_timescales = self.n_dim // 2
#         super().__init__(**kwargs)

#     def get_timescales(self):
#         # This is a bit hacky, but works
#         timescales = self.max_time ** np.linspace(0, 1, self._num_timescales)
#         return timescales

#     def build(self, input_shape):
#         assert len(input_shape) == 3
#         self.timescales = self.add_weight(
#             'timescales',
#             (self._num_timescales, ),
#             trainable=False,
#             initializer=tf.keras.initializers.Constant(self.get_timescales())
#         )

#     def __call__(self, times):
#         scaled_time = times / self.timescales[None, None, :]
#         signal = tf.concat(
#             [
#                 tf.sin(scaled_time),
#                 tf.cos(scaled_time)
#             ],
#             axis=-1)
#         return signal

#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], input_shape[1], self.n_dim)

In [ ]:
  
class CVE(Layer):
    def __init__(self, hid_units, output_dim):
        self.hid_units = hid_units
        self.output_dim = output_dim
        super(CVE, self).__init__()
        
    def build(self, input_shape): 
        self.W1 = self.add_weight(name='CVE_W1',
                            shape=(1, self.hid_units),
                            initializer='glorot_uniform',
                            trainable=True)
        self.b1 = self.add_weight(name='CVE_b1',
                            shape=(self.hid_units,),
                            initializer='zeros',
                            trainable=True)
        self.W2 = self.add_weight(name='CVE_W2',
                            shape=(self.hid_units, self.output_dim),
                            initializer='glorot_uniform',
                            trainable=True)
        super(CVE, self).build(input_shape)
        
    def call(self, x):
        x = K.expand_dims(x, axis=-1)
        x = K.dot(K.tanh(K.bias_add(K.dot(x, self.W1), self.b1)), self.W2)
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape + (self.output_dim,)
    
    
class Attention(Layer):
    
    def __init__(self, hid_dim):
        self.hid_dim = hid_dim
        super(Attention, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        self.W = self.add_weight(shape=(d, self.hid_dim), name='Att_W',
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.hid_dim,), name='Att_b',
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(shape=(self.hid_dim,1), name='Att_u',
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(Attention, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e30):
        attn_weights = K.dot(K.tanh(K.bias_add(K.dot(x,self.W), self.b)), self.u)
        mask = K.expand_dims(mask, axis=-1)
        attn_weights = mask*attn_weights + (1-mask)*mask_value
        attn_weights = K.softmax(attn_weights, axis=-2)
        return attn_weights
        
    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (1,)
    
    
class Transformer(Layer):
    
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0):
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = K.epsilon() * K.epsilon()
        super(Transformer, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        if self.dk==None:
            self.dk = d//self.h
        if self.dv==None:
            self.dv = d//self.h
        if self.dff==None:
            self.dff = 2*d
        self.Wq = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wq',
                                 initializer='glorot_uniform', trainable=True)
        self.Wk = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wk',
                                 initializer='glorot_uniform', trainable=True)
        self.Wv = self.add_weight(shape=(self.N, self.h, d, self.dv), name='Wv',
                                 initializer='glorot_uniform', trainable=True)
        self.Wo = self.add_weight(shape=(self.N, self.dv*self.h, d), name='Wo',
                                 initializer='glorot_uniform', trainable=True)
        self.W1 = self.add_weight(shape=(self.N, d, self.dff), name='W1',
                                 initializer='glorot_uniform', trainable=True)
        self.b1 = self.add_weight(shape=(self.N, self.dff), name='b1',
                                 initializer='zeros', trainable=True)
        self.W2 = self.add_weight(shape=(self.N, self.dff, d), name='W2',
                                 initializer='glorot_uniform', trainable=True)
        self.b2 = self.add_weight(shape=(self.N, d), name='b2',
                                 initializer='zeros', trainable=True)
        self.gamma = self.add_weight(shape=(2*self.N,), name='gamma',
                                 initializer='ones', trainable=True)
        self.beta = self.add_weight(shape=(2*self.N,), name='beta',
                                 initializer='zeros', trainable=True)
        super(Transformer, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e-30):
        mask = K.expand_dims(mask, axis=-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = K.dot(x, self.Wq[i,j,:,:])
                k = K.permute_dimensions(K.dot(x, self.Wk[i,j,:,:]), (0,2,1))
                v = K.dot(x, self.Wv[i,j,:,:])
                A = K.batch_dot(q,k)
                # Mask unobserved steps.
                A = mask*A + (1-mask)*mask_value
                # Mask for attention dropout.
                def dropped_A():
                    dp_mask = K.cast((K.random_uniform(shape=array_ops.shape(A))>=self.dropout), K.floatx())
                    return A*dp_mask + (1-dp_mask)*mask_value
                A = sc.smart_cond(K.learning_phase(), dropped_A, lambda: array_ops.identity(A))
                A = K.softmax(A, axis=-1)
                mha_ops.append(K.batch_dot(A,v))
            conc = K.concatenate(mha_ops, axis=-1)
            proj = K.dot(conc, self.Wo[i,:,:])
            # Dropout.
            proj = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(proj, rate=self.dropout)),\
                                       lambda: array_ops.identity(proj))
            # Add & LN
            x = x+proj
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i] + self.beta[2*i]
            # FFN
            ffn_op = K.bias_add(K.dot(K.relu(K.bias_add(K.dot(x, self.W1[i,:,:]), self.b1[i,:])), 
                           self.W2[i,:,:]), self.b2[i,:,])
            # Dropout.
            ffn_op = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(ffn_op, rate=self.dropout)),\
                                       lambda: array_ops.identity(ffn_op))
            # Add & LN
            x = x+ffn_op
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i+1] + self.beta[2*i+1]            
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape


def build_strats(D, max_len, V, d, N, he, dropout, forecast=False):
    # demo
    demo = Input(shape=(D,))
    demo_enc = Dense(2*d, activation='tanh')(demo)
    demo_enc = Dense(d, activation='tanh')(demo_enc)
    
    ## text
    # text
    texts = Input(shape=(768*50,))
#     text_enc = Dense(1000, activation='relu')(texts)
#     text_enc = Dense(d, activation='relu')(text_enc)

    text_2d = Lambda(lambda x: K.reshape(x, (-1, 50, 768)), input_shape=(None, 50*768))(texts)
    
    # Q 
    text_d = Dense(50, activation='relu')(texts)
    
    
    # text time
    text_times = Input(shape=(50,))
    cve_units = int(np.sqrt(d))
    text_times_emb = CVE(cve_units, 768)(text_times)
    
    # text varis
    text_varis = Input(shape=(50,))
    # text_varis_emb = Embedding(V+1, d)(text_varis)
    
    text_comb_emb = Add()([text_2d, text_times_emb])
    text_mask = Lambda(lambda x:K.clip(x,0,1))(text_varis)
    # text_mask =  Lambda(lambda x: K.reshape(x, (-1, 50, 768)), input_shape=(None, 50*768))(text_varis)
    text_cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(text_comb_emb, mask=text_mask)
    text_attn_weights = Attention(2*d)(text_cont_emb, mask=text_mask)
    text_fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([text_cont_emb, text_attn_weights])
    
    text_dense = Dense(100, activation='relu')(text_fused_emb)
    
    
    ## physio
    # triplet
    varis = Input(shape=(max_len,))
    values = Input(shape=(max_len,))
    times = Input(shape=(max_len,))
    
    
    varis_emb = Embedding(V+1, d)(varis)
    cve_units = int(np.sqrt(d))
    values_emb = CVE(cve_units, d)(values)
    times_emb = CVE(cve_units, d)(times)
    
    
    # comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d
    
    # comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d
    
    # Q
    comb_emb = Add()([varis_emb, values_emb, times_emb, text_d])
#     demo_enc = Lambda(lambda x:K.expand_dims(x, axis=-2))(demo_enc) # b, 1, d
#     comb_emb = Concatenate(axis=-2)([demo_enc, comb_emb]) # b, L+1, d
    mask = Lambda(lambda x:K.clip(x,0,1))(varis) # b, L
#     mask = Lambda(lambda x:K.concatenate((K.ones_like(x)[:,0:1], x), axis=-1))(mask) # b, L+1
    cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(comb_emb, mask=mask)
    attn_weights = Attention(2*d)(cont_emb, mask=mask)
    fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, attn_weights])
    
#     # embed text input
#     texts = Input(shape=(33792,))
#     text_enc = Dense(1000, activation='relu')(texts)
#     text_enc = Dense(d, activation='relu')(text_enc)


    # conc = Concatenate(axis=-1)([fused_emb, text_fused_emb, demo_enc])
    conc = Concatenate(axis=-1)([fused_emb, text_dense, demo_enc])
    
    
#     fore_op = Dense(V)(conc)
#     op = Dense(1, activation='sigmoid')(fore_op)
#     model = Model([demo, times, values, varis, texts, text_times, text_varis], op)
#     if forecast:
#         fore_model = Model([demo, times, values, varis, texts, text_times, text_varis], fore_op)
#         return [model, fore_model]
#     return model

    
    fore_op = Dense(V)(conc)
    op = Dense(1, activation='sigmoid')(fore_op)
    model = Model([demo, times, values, varis, texts, text_times, text_varis], op)
    if forecast:
        fore_model = Model([demo, times, values, varis, texts, text_times, text_varis], fore_op)
        return [model, fore_model]
    return model

# To tune:
# 1. Transformer parameters. (N, h, dropout)
# 2. Normalization

## Pretrain on forecasting

In [1]:
1e-4

0.0001

In [ ]:
lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 5
# lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
d, N, he, dropout = 50, 2, 4, 0.2
model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
print (fore_model.summary())
fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))

# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)

fore_savepath = 'hierar/models/forecasting/forecasting_1_epochs.h5'


# save losses for visualization
# val_losses = []
train_losses = []

# for e in range(1000):
#     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
#     e_loss = 0
#     pbar = tqdm(range(0, len(e_indices), batch_size))
#     for start in pbar:
#         ind = e_indices[start:start+batch_size]
#         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
#         pbar.set_description('%f'%(e_loss/(start+1)))
#     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     val_losses.append(val_loss)
#     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
#     train_losses.append(e_loss*batch_size/samples_per_epoch)
#     # model should be saved here after each epoch in case of unexpected disconnection
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         fore_model.save_weights(fore_savepath)
#         best_epoch = e
#     if (e-best_epoch)>patience:
#         break

for e in range(1):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
        pbar.set_description('%f'%(e_loss/(start+1)))
    # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
    # val_losses.append(val_loss)
    # print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
    print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch)
    train_losses.append(e_loss*batch_size/samples_per_epoch)
    fore_model.save_weights(fore_savepath)
    # # model should be saved here after each epoch in case of unexpected disconnection
    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss
    #     fore_model.save_weights(fore_savepath)
    #     best_epoch = e
    # if (e-best_epoch)>patience:
    #     break
    print(train_losses)

In [ ]:
# train_losses

In [ ]:
# val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# print(f'validation loss: {val_loss}')

In [ ]:
# val_loss
## 8.26081371307373

In [ ]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 5
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())


# # Pretrain fore_model.
# best_val_loss = np.inf
# N_fore = len(fore_train_op)

# # save losses for visualization
# # val_losses = []
# train_losses = [9.6726073936373]

# # for e in range(1000):
# #     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
# #     e_loss = 0
# #     pbar = tqdm(range(0, len(e_indices), batch_size))
# #     for start in pbar:
# #         ind = e_indices[start:start+batch_size]
# #         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
# #         pbar.set_description('%f'%(e_loss/(start+1)))
# #     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# #     val_losses.append(val_loss)
# #     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
# #     train_losses.append(e_loss*batch_size/samples_per_epoch)
# #     # model should be saved here after each epoch in case of unexpected disconnection
# #     if val_loss < best_val_loss:
# #         best_val_loss = val_loss
# #         fore_model.save_weights(fore_savepath)
# #         best_epoch = e
# #     if (e-best_epoch)>patience:
# #         break

# for e in range(100):

#     fore_loadpath = 'hierar/models/forecasting/forecasting_'+str(e+1)+'_epochs.h5'
#     fore_savepath = 'hierar/models/forecasting/forecasting_'+str(e+1+1)+'_epochs.h5'
    
#     fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#     fore_model.load_weights(fore_loadpath)

#     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
#     e_loss = 0
#     pbar = tqdm(range(0, len(e_indices), batch_size))
#     for start in pbar:
#         ind = e_indices[start:start+batch_size]
#         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
#         pbar.set_description('%f'%(e_loss/(start+1)))
#     # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     # val_losses.append(val_loss)
#     # print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
#     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch)
#     train_losses.append(e_loss*batch_size/samples_per_epoch)
#     fore_model.save_weights(fore_savepath)
#     # # model should be saved here after each epoch in case of unexpected disconnection
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     fore_model.save_weights(fore_savepath)
#     #     best_epoch = e
#     # if (e-best_epoch)>patience:
#     #     break
#     print(train_losses)

    
# # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# # print(f'validation loss: {val_loss}')

In [ ]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 5
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())


# # Pretrain fore_model.
# best_val_loss = np.inf
# N_fore = len(fore_train_op)

# # save losses for visualization
# # val_losses = []
# train_losses = [9.6726073936373, 7.827818336114287, 7.339528672397137, 7.14841083496809, 6.650257249735295, 6.71930052049458, 6.3753932047262785, 6.246122511550784, 6.383257875964046, 6.14440975420177, 6.1410007086768745, 5.9775916450470685, 5.9476286797598, 6.023135863617062, 5.962186483182013, 5.85297311808914, 5.768680865727365, 5.801690417081118]

# # for e in range(1000):
# #     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
# #     e_loss = 0
# #     pbar = tqdm(range(0, len(e_indices), batch_size))
# #     for start in pbar:
# #         ind = e_indices[start:start+batch_size]
# #         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
# #         pbar.set_description('%f'%(e_loss/(start+1)))
# #     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# #     val_losses.append(val_loss)
# #     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
# #     train_losses.append(e_loss*batch_size/samples_per_epoch)
# #     # model should be saved here after each epoch in case of unexpected disconnection
# #     if val_loss < best_val_loss:
# #         best_val_loss = val_loss
# #         fore_model.save_weights(fore_savepath)
# #         best_epoch = e
# #     if (e-best_epoch)>patience:
# #         break

# for e in range(100):

#     fore_loadpath = 'hierar/models/forecasting/forecasting_'+str(e+17)+'_epochs.h5'
#     fore_savepath = 'hierar/models/forecasting/forecasting_'+str(e+17+1)+'_epochs.h5'
    
#     fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#     fore_model.load_weights(fore_loadpath)

#     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
#     e_loss = 0
#     pbar = tqdm(range(0, len(e_indices), batch_size))
#     for start in pbar:
#         ind = e_indices[start:start+batch_size]
#         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
#         pbar.set_description('%f'%(e_loss/(start+1)))
#     # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     # val_losses.append(val_loss)
#     # print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
#     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch)
#     train_losses.append(e_loss*batch_size/samples_per_epoch)
#     fore_model.save_weights(fore_savepath)
#     # # model should be saved here after each epoch in case of unexpected disconnection
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     fore_model.save_weights(fore_savepath)
#     #     best_epoch = e
#     # if (e-best_epoch)>patience:
#     #     break
#     print(train_losses)

    
# # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# # print(f'validation loss: {val_loss}')

In [ ]:
# train_losses = [9.6726073936373,
#  7.827818336114287,
#  7.339528672397137,
#  7.14841083496809,
#  6.650257249735295,
#  6.71930052049458,
#  6.3753932047262785,
#  6.246122511550784,
#  6.383257875964046,
#  6.14440975420177,
#  6.1410007086768745,
#  5.9775916450470685,
#  5.9476286797598,
#  6.023135863617062,
#  5.962186483182013,
#  5.85297311808914,
#  5.768680865727365,
#  5.801690417081118,
#  5.801871319301426,
#  5.561956452168524,
#  5.708363916277886,
#  5.792748096622526,
#  5.7496581207588315,
#  5.439435611739754,
#  5.584537696801126,
#  5.704166349992156,
#  5.440190438069403,
#  5.513703255839646,
#  5.608548376485706,
#  5.677496524602175,
#  5.360032863207161,
#  5.6003397961705925,
#  5.352133833542466,
#  5.418496854007244,
#  5.611297566629946,
#  5.386051201373339,
#  5.472518955282867,
#  5.562354943752289,
#  5.387425537928939,
#  5.473685835413635,
#  5.40586627010256,
#  5.478820224441588,
#  5.492045880965889,
#  5.6129028406366706,
#  5.312044258899987,
#  5.362207580134273,
#  5.408285063430667,
#  5.408482057116926,
#  5.348271051608026,
#  5.3705539362505075,
#  5.359316195063293,
#  5.4364967308565975,
#  5.388157960027456,
#  5.397286337986588,
#  5.3096947432309385,
#  5.544507872723043,
#  5.372893713116646,
#  5.236083977781236,
#  5.232814045846462,
#  5.388364875204861,
#  5.285311912223697,
#  5.390774428062141,
#  5.12165357824415,
#  5.389126623794437,
#  5.371281063705683,
#  5.263771855421364]

In [ ]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 5
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())


# # Pretrain fore_model.
# best_val_loss = np.inf
# N_fore = len(fore_train_op)

# # save losses for visualization
# # val_losses = []

# # for e in range(1000):
# #     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
# #     e_loss = 0
# #     pbar = tqdm(range(0, len(e_indices), batch_size))
# #     for start in pbar:
# #         ind = e_indices[start:start+batch_size]
# #         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
# #         pbar.set_description('%f'%(e_loss/(start+1)))
# #     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# #     val_losses.append(val_loss)
# #     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
# #     train_losses.append(e_loss*batch_size/samples_per_epoch)
# #     # model should be saved here after each epoch in case of unexpected disconnection
# #     if val_loss < best_val_loss:
# #         best_val_loss = val_loss
# #         fore_model.save_weights(fore_savepath)
# #         best_epoch = e
# #     if (e-best_epoch)>patience:
# #         break

# for e in range(100):

#     fore_loadpath = 'hierar/models/forecasting/forecasting_'+str(e+66)+'_epochs.h5'
#     fore_savepath = 'hierar/models/forecasting/forecasting_'+str(e+66+1)+'_epochs.h5'
    
#     fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#     fore_model.load_weights(fore_loadpath)

#     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
#     e_loss = 0
#     pbar = tqdm(range(0, len(e_indices), batch_size))
#     for start in pbar:
#         ind = e_indices[start:start+batch_size]
#         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
#         pbar.set_description('%f'%(e_loss/(start+1)))
#     # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     # val_losses.append(val_loss)
#     # print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
#     print ('Epoch', e+66, 'loss', e_loss*batch_size/samples_per_epoch)
#     train_losses.append(e_loss*batch_size/samples_per_epoch)
#     fore_model.save_weights(fore_savepath)
#     # # model should be saved here after each epoch in case of unexpected disconnection
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     fore_model.save_weights(fore_savepath)
#     #     best_epoch = e
#     # if (e-best_epoch)>patience:
#     #     break
#     print(train_losses)

    
# # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# # print(f'validation loss: {val_loss}')

In [ ]:
# train_losses = [9.6726073936373, 7.827818336114287, 7.339528672397137, 7.14841083496809, 6.650257249735295, 6.71930052049458, 6.3753932047262785, 6.246122511550784, 6.383257875964046, 6.14440975420177, 6.1410007086768745, 5.9775916450470685, 5.9476286797598, 6.023135863617062, 5.962186483182013, 5.85297311808914, 5.768680865727365, 5.801690417081118, 5.801871319301426, 5.561956452168524, 5.708363916277886, 5.792748096622526, 5.7496581207588315, 5.439435611739754, 5.584537696801126, 5.704166349992156, 5.440190438069403, 5.513703255839646, 5.608548376485706, 5.677496524602175, 5.360032863207161, 5.6003397961705925, 5.352133833542466, 5.418496854007244, 5.611297566629946, 5.386051201373339, 5.472518955282867, 5.562354943752289, 5.387425537928939, 5.473685835413635, 5.40586627010256, 5.478820224441588, 5.492045880965889, 5.6129028406366706, 5.312044258899987, 5.362207580134273, 5.408285063430667, 5.408482057116926, 5.348271051608026, 5.3705539362505075, 5.359316195063293, 5.4364967308565975, 5.388157960027456, 5.397286337986588, 5.3096947432309385, 5.544507872723043, 5.372893713116646, 5.236083977781236, 5.232814045846462, 5.388364875204861, 5.285311912223697, 5.390774428062141, 5.12165357824415, 5.389126623794437, 5.371281063705683, 5.263771855421364, 5.23746578168124, 5.416688159443438, 5.205470519624651, 5.297770969420672, 5.294283067695797, 5.591666744612158, 5.133070289082825, 5.315817803628743, 5.261407737210393, 5.284937768243253, 5.29699768114835, 5.132637087032199, 5.35254779919982, 5.23261724255979, 5.149450082182884, 5.167635361738503, 5.311890206597745, 5.282972572669387, 5.1295755824819205, 5.075229794420302, 5.2687539759278295, 5.233767274618149, 5.200219418369233, 5.249976786412299, 5.303496397100389, 5.187022073045373, 5.205709896497428, 5.231979852579534, 5.264038751870394, 5.181498194336891, 5.148470848277211, 5.304017275571823, 5.230564057789743, 5.12524469319731, 5.221126203387976, 5.1808790271729235, 5.086285249628126, 5.1751138539239765, 5.151869556121528, 5.214914522059262, 5.275770538896322, 5.072343826666474, 5.187715657204389, 5.208328050747514, 5.273546916656196, 5.181960097290576, 5.268980304412544, 5.200946623757481, 5.225574138015508, 5.215654050521553, 5.1830766187980775, 5.251943679004907, 5.327554960735142, 5.2131019575148825, 5.169291452169419]

In [ ]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 5
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())


# # Pretrain fore_model.
# best_val_loss = np.inf
# N_fore = len(fore_train_op)

# # save losses for visualization
# # val_losses = []

# # for e in range(1000):
# #     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
# #     e_loss = 0
# #     pbar = tqdm(range(0, len(e_indices), batch_size))
# #     for start in pbar:
# #         ind = e_indices[start:start+batch_size]
# #         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
# #         pbar.set_description('%f'%(e_loss/(start+1)))
# #     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# #     val_losses.append(val_loss)
# #     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
# #     train_losses.append(e_loss*batch_size/samples_per_epoch)
# #     # model should be saved here after each epoch in case of unexpected disconnection
# #     if val_loss < best_val_loss:
# #         best_val_loss = val_loss
# #         fore_model.save_weights(fore_savepath)
# #         best_epoch = e
# #     if (e-best_epoch)>patience:
# #         break

# for e in range(100):

#     fore_loadpath = 'hierar/models/forecasting/forecasting_'+str(e+121)+'_epochs.h5'
#     fore_savepath = 'hierar/models/forecasting/forecasting_'+str(e+121+1)+'_epochs.h5'
    
#     fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#     fore_model.load_weights(fore_loadpath)

#     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
#     e_loss = 0
#     pbar = tqdm(range(0, len(e_indices), batch_size))
#     for start in pbar:
#         ind = e_indices[start:start+batch_size]
#         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
#         pbar.set_description('%f'%(e_loss/(start+1)))
#     # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     # val_losses.append(val_loss)
#     # print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
#     print ('Epoch', e+66, 'loss', e_loss*batch_size/samples_per_epoch)
#     train_losses.append(e_loss*batch_size/samples_per_epoch)
#     fore_model.save_weights(fore_savepath)
#     # # model should be saved here after each epoch in case of unexpected disconnection
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     fore_model.save_weights(fore_savepath)
#     #     best_epoch = e
#     # if (e-best_epoch)>patience:
#     #     break
#     print(train_losses)

    
# # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# # print(f'validation loss: {val_loss}')

In [ ]:
train_losses = [9.6726073936373, 7.827818336114287, 7.339528672397137, 7.14841083496809, 6.650257249735295, 6.71930052049458, 6.3753932047262785, 6.246122511550784, 6.383257875964046, 6.14440975420177, 6.1410007086768745, 5.9775916450470685, 5.9476286797598, 6.023135863617062, 5.962186483182013, 5.85297311808914, 5.768680865727365, 5.801690417081118, 5.801871319301426, 5.561956452168524, 5.708363916277886, 5.792748096622526, 5.7496581207588315, 5.439435611739754, 5.584537696801126, 5.704166349992156, 5.440190438069403, 5.513703255839646, 5.608548376485706, 5.677496524602175, 5.360032863207161, 5.6003397961705925, 5.352133833542466, 5.418496854007244, 5.611297566629946, 5.386051201373339, 5.472518955282867, 5.562354943752289, 5.387425537928939, 5.473685835413635, 5.40586627010256, 5.478820224441588, 5.492045880965889, 5.6129028406366706, 5.312044258899987, 5.362207580134273, 5.408285063430667, 5.408482057116926, 5.348271051608026, 5.3705539362505075, 5.359316195063293, 5.4364967308565975, 5.388157960027456, 5.397286337986588, 5.3096947432309385, 5.544507872723043, 5.372893713116646, 5.236083977781236, 5.232814045846462, 5.388364875204861, 5.285311912223697, 5.390774428062141, 5.12165357824415, 5.389126623794437, 5.371281063705683, 5.263771855421364, 5.23746578168124, 5.416688159443438, 5.205470519624651, 5.297770969420672, 5.294283067695797, 5.591666744612158, 5.133070289082825, 5.315817803628743, 5.261407737210393, 5.284937768243253, 5.29699768114835, 5.132637087032199, 5.35254779919982, 5.23261724255979, 5.149450082182884, 5.167635361738503, 5.311890206597745, 5.282972572669387, 5.1295755824819205, 5.075229794420302, 5.2687539759278295, 5.233767274618149, 5.200219418369233, 5.249976786412299, 5.303496397100389, 5.187022073045373, 5.205709896497428, 5.231979852579534, 5.264038751870394, 5.181498194336891, 5.148470848277211, 5.304017275571823, 5.230564057789743, 5.12524469319731, 5.221126203387976, 5.1808790271729235, 5.086285249628126, 5.1751138539239765, 5.151869556121528, 5.214914522059262, 5.275770538896322, 5.072343826666474, 5.187715657204389, 5.208328050747514, 5.273546916656196, 5.181960097290576, 5.268980304412544, 5.200946623757481, 5.225574138015508, 5.215654050521553, 5.1830766187980775, 5.251943679004907, 5.327554960735142, 5.2131019575148825, 5.169291452169419, 5.136195200160146, 5.244120051041246, 5.070332445763052, 5.19596264667809, 5.064282140806317, 5.349931869506836]

In [ ]:
lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 5
# lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
d, N, he, dropout = 50, 2, 4, 0.2
model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
print (fore_model.summary())


# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)

# save losses for visualization
# val_losses = []

# for e in range(1000):
#     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
#     e_loss = 0
#     pbar = tqdm(range(0, len(e_indices), batch_size))
#     for start in pbar:
#         ind = e_indices[start:start+batch_size]
#         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
#         pbar.set_description('%f'%(e_loss/(start+1)))
#     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     val_losses.append(val_loss)
#     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
#     train_losses.append(e_loss*batch_size/samples_per_epoch)
#     # model should be saved here after each epoch in case of unexpected disconnection
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         fore_model.save_weights(fore_savepath)
#         best_epoch = e
#     if (e-best_epoch)>patience:
#         break

for e in range(100):

    fore_loadpath = 'hierar/models/forecasting/forecasting_'+str(e+127)+'_epochs.h5'
    fore_savepath = 'hierar/models/forecasting/forecasting_'+str(e+127+1)+'_epochs.h5'
    
    fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
    fore_model.load_weights(fore_loadpath)

    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
        pbar.set_description('%f'%(e_loss/(start+1)))
    # val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
    # val_losses.append(val_loss)
    # print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
    print ('Epoch', e+66, 'loss', e_loss*batch_size/samples_per_epoch)
    train_losses.append(e_loss*batch_size/samples_per_epoch)
    fore_model.save_weights(fore_savepath)
    # # model should be saved here after each epoch in case of unexpected disconnection
    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss
    #     fore_model.save_weights(fore_savepath)
    #     best_epoch = e
    # if (e-best_epoch)>patience:
    #     break
    print(train_losses)

    
# val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
# print(f'validation loss: {val_loss}')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 38400)]      0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 880)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 880)]        0           []                               
                                                                                            

0.159728: 100%|██████████| 3200/3200 [04:12<00:00, 12.65it/s]


Epoch 66 loss 5.109742404706776
[9.6726073936373, 7.827818336114287, 7.339528672397137, 7.14841083496809, 6.650257249735295, 6.71930052049458, 6.3753932047262785, 6.246122511550784, 6.383257875964046, 6.14440975420177, 6.1410007086768745, 5.9775916450470685, 5.9476286797598, 6.023135863617062, 5.962186483182013, 5.85297311808914, 5.768680865727365, 5.801690417081118, 5.801871319301426, 5.561956452168524, 5.708363916277886, 5.792748096622526, 5.7496581207588315, 5.439435611739754, 5.584537696801126, 5.704166349992156, 5.440190438069403, 5.513703255839646, 5.608548376485706, 5.677496524602175, 5.360032863207161, 5.6003397961705925, 5.352133833542466, 5.418496854007244, 5.611297566629946, 5.386051201373339, 5.472518955282867, 5.562354943752289, 5.387425537928939, 5.473685835413635, 5.40586627010256, 5.478820224441588, 5.492045880965889, 5.6129028406366706, 5.312044258899987, 5.362207580134273, 5.408285063430667, 5.408482057116926, 5.348271051608026, 5.3705539362505075, 5.359316195063293, 

0.161364: 100%|██████████| 3200/3200 [04:00<00:00, 13.28it/s]


Epoch 67 loss 5.162085954248905
[9.6726073936373, 7.827818336114287, 7.339528672397137, 7.14841083496809, 6.650257249735295, 6.71930052049458, 6.3753932047262785, 6.246122511550784, 6.383257875964046, 6.14440975420177, 6.1410007086768745, 5.9775916450470685, 5.9476286797598, 6.023135863617062, 5.962186483182013, 5.85297311808914, 5.768680865727365, 5.801690417081118, 5.801871319301426, 5.561956452168524, 5.708363916277886, 5.792748096622526, 5.7496581207588315, 5.439435611739754, 5.584537696801126, 5.704166349992156, 5.440190438069403, 5.513703255839646, 5.608548376485706, 5.677496524602175, 5.360032863207161, 5.6003397961705925, 5.352133833542466, 5.418496854007244, 5.611297566629946, 5.386051201373339, 5.472518955282867, 5.562354943752289, 5.387425537928939, 5.473685835413635, 5.40586627010256, 5.478820224441588, 5.492045880965889, 5.6129028406366706, 5.312044258899987, 5.362207580134273, 5.408285063430667, 5.408482057116926, 5.348271051608026, 5.3705539362505075, 5.359316195063293, 

...

0.160913: 100%|██████████| 3200/3200 [03:54<00:00, 13.67it/s]


Epoch 101 loss 5.147645687051117
[9.6726073936373, 7.827818336114287, 7.339528672397137, 7.14841083496809, 6.650257249735295, 6.71930052049458, 6.3753932047262785, 6.246122511550784, 6.383257875964046, 6.14440975420177, 6.1410007086768745, 5.9775916450470685, 5.9476286797598, 6.023135863617062, 5.962186483182013, 5.85297311808914, 5.768680865727365, 5.801690417081118, 5.801871319301426, 5.561956452168524, 5.708363916277886, 5.792748096622526, 5.7496581207588315, 5.439435611739754, 5.584537696801126, 5.704166349992156, 5.440190438069403, 5.513703255839646, 5.608548376485706, 5.677496524602175, 5.360032863207161, 5.6003397961705925, 5.352133833542466, 5.418496854007244, 5.611297566629946, 5.386051201373339, 5.472518955282867, 5.562354943752289, 5.387425537928939, 5.473685835413635, 5.40586627010256, 5.478820224441588, 5.492045880965889, 5.6129028406366706, 5.312044258899987, 5.362207580134273, 5.408285063430667, 5.408482057116926, 5.348271051608026, 5.3705539362505075, 5.359316195063293,

0.159618: 100%|██████████| 3200/3200 [03:53<00:00, 13.70it/s]


Epoch 102 loss 5.106238405890763
[9.6726073936373, 7.827818336114287, 7.339528672397137, 7.14841083496809, 6.650257249735295, 6.71930052049458, 6.3753932047262785, 6.246122511550784, 6.383257875964046, 6.14440975420177, 6.1410007086768745, 5.9775916450470685, 5.9476286797598, 6.023135863617062, 5.962186483182013, 5.85297311808914, 5.768680865727365, 5.801690417081118, 5.801871319301426, 5.561956452168524, 5.708363916277886, 5.792748096622526, 5.7496581207588315, 5.439435611739754, 5.584537696801126, 5.704166349992156, 5.440190438069403, 5.513703255839646, 5.608548376485706, 5.677496524602175, 5.360032863207161, 5.6003397961705925, 5.352133833542466, 5.418496854007244, 5.611297566629946, 5.386051201373339, 5.472518955282867, 5.562354943752289, 5.387425537928939, 5.473685835413635, 5.40586627010256, 5.478820224441588, 5.492045880965889, 5.6129028406366706, 5.312044258899987, 5.362207580134273, 5.408285063430667, 5.408482057116926, 5.348271051608026, 5.3705539362505075, 5.359316195063293,

0.157883:  48%|████▊     | 1547/3200 [01:59<01:52, 14.70it/s]

In [1]:
len([9.6726073936373, 7.827818336114287, 7.339528672397137, 7.14841083496809, 6.650257249735295, 6.71930052049458, 6.3753932047262785, 6.246122511550784, 6.383257875964046, 6.14440975420177, 6.1410007086768745, 5.9775916450470685, 5.9476286797598, 6.023135863617062, 5.962186483182013, 5.85297311808914, 5.768680865727365, 5.801690417081118, 5.801871319301426, 5.561956452168524, 5.708363916277886, 5.792748096622526, 5.7496581207588315, 5.439435611739754, 5.584537696801126, 5.704166349992156, 5.440190438069403, 5.513703255839646, 5.608548376485706, 5.677496524602175, 5.360032863207161, 5.6003397961705925, 5.352133833542466, 5.418496854007244, 5.611297566629946, 5.386051201373339, 5.472518955282867, 5.562354943752289, 5.387425537928939, 5.473685835413635, 5.40586627010256, 5.478820224441588, 5.492045880965889, 5.6129028406366706, 5.312044258899987, 5.362207580134273, 5.408285063430667, 5.408482057116926, 5.348271051608026, 5.3705539362505075, 5.359316195063293, 5.4364967308565975, 5.388157960027456, 5.397286337986588, 5.3096947432309385, 5.544507872723043, 5.372893713116646, 5.236083977781236, 5.232814045846462, 5.388364875204861, 5.285311912223697, 5.390774428062141, 5.12165357824415, 5.389126623794437, 5.371281063705683, 5.263771855421364, 5.23746578168124, 5.416688159443438, 5.205470519624651, 5.297770969420672, 5.294283067695797, 5.591666744612158, 5.133070289082825, 5.315817803628743, 5.261407737210393, 5.284937768243253, 5.29699768114835, 5.132637087032199, 5.35254779919982, 5.23261724255979, 5.149450082182884, 5.167635361738503, 5.311890206597745, 5.282972572669387, 5.1295755824819205, 5.075229794420302, 5.2687539759278295, 5.233767274618149, 5.200219418369233, 5.249976786412299, 5.303496397100389, 5.187022073045373, 5.205709896497428, 5.231979852579534, 5.264038751870394, 5.181498194336891, 5.148470848277211, 5.304017275571823, 5.230564057789743, 5.12524469319731, 5.221126203387976, 5.1808790271729235, 5.086285249628126, 5.1751138539239765, 5.151869556121528, 5.214914522059262, 5.275770538896322, 5.072343826666474, 5.187715657204389, 5.208328050747514, 5.273546916656196, 5.181960097290576, 5.268980304412544, 5.200946623757481, 5.225574138015508, 5.215654050521553, 5.1830766187980775, 5.251943679004907, 5.327554960735142, 5.2131019575148825, 5.169291452169419, 5.136195200160146, 5.244120051041246, 5.070332445763052, 5.19596264667809, 5.064282140806317, 5.349931869506836, 5.109742404706776, 5.162085954248905, 4.956474395804107, 5.151966934166849, 5.298011938072741, 5.2769091617316, 5.139627133905887, 5.085182873085142, 5.180812531858683, 5.322513702511787, 5.0061469754949215, 5.069373594857752, 5.017210310138762, 5.104229423068464, 5.27019073240459, 5.165618509575725, 5.084191198311746, 5.10948396999389, 5.165602688938379, 5.254258686006069, 5.263038488291204, 5.133429842889309, 5.294457230269909, 5.154379370212555, 5.186611767858267, 5.337653120085597, 5.178275236412883, 5.216593872718513, 5.173797446973622, 5.11946918297559, 5.1708628893271085, 5.179464494660497, 5.312201052866876, 5.166050132624805, 5.260845541916788, 5.147645687051117, 5.106238405890763])

164